___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Freddy Dratwa

#### Victor Vazquez

#### Michel Haggiag

#### Allan Singer

#### Ricardo Ferraro

___

# Estudo do preço do carro

___

## A. INTRODUÇÃO

## https://www.kaggle.com/jpayne/852k-used-car-listings

O objetivo deste projeto é chegar ao melhor modelo possível que seja capaz de prever o preço de um carro usado nos EUA a partir das características presentes no dataset:
    - Preço;
    - Ano;
    - Quilometragem;
    - Cidade;
    - Estado;
    - Marca;
    - Modelo;
    - Número de Chassis.

A base de dados possui 1.2 milhão de carros com data de produção entre os anos 1997 a 2018 de 58 marcas diferentes, sendo estes 2914 modelos distintos com o preço mÍnimo de US1.500,00 e máximo de US462.000,00 , com a mínima quilometragem 8 Km e a máxima 2.286.260,8 Km.<br>

Como uma primeira previsão, imagina-se que algumas das características do dataset influenciam mais no preço do carro do que outras. Por exemplo, é de se esperar que um carro comprado há mais tempo tenha seu preço menor atualmente. Ou seja, a variavel "Ano", que representa o ano em que o carro foi fabricado, deve influenciar muito no preço do veículo, bem como a quilometragem, onde um carro que foi comprado mas usado pouco deve permanecer mais barato do que um usado muitas vezes.<br>

Pelo outro lado, imagina-se que a variável "Cidade", a qual representa a cidade onde o carro foi comprado, não tenha tanta influência no preço do veículo, a não ser que comparadas  duas cidades com condições muito diferentes. Por exemplo, como se tratam de cidades apenas nos EUA, imaina-se que em cidades no litoral, os carros se depreciem com velocidade maior, como consequência do fenômeno da Maresia.<br>

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

### BASE DE DADOS

**Limpando e tratando dataset**

Nas células abaixo, o dataset é limpo e reorganiazdo. Para se fazer isso, a coluna "Chassi" é removoida, pois esta é uma característica individual para cada veículo e não influencia no preço final do mesmo. Alem disso, duas colunas vazias foram reitradas. Então, a variável milhagem foi transformada em quilometragem, multplicando seu valor por 1.6. Por último, a variavel ano foi transformada em idade, ou seja, a diferença do ano atual (2019) para o ano em que o carro foi vendido. 

In [2]:
dataset = pd.read_csv("Dataframe.csv")

In [3]:
if "Vin" in dataset:
    dataset = dataset.drop(columns=["Vin", "Unnamed: 9", "Unnamed: 10", "Unnamed: 0"])
    dataset["Mileage"] = dataset["Mileage"].apply(lambda x: x*1.6)

    dataset.columns = ["ID","PRECO","ANO","KM","CIDADE","ESTADO","MARCA","MODELO"]
    dataset["IDADE"] = dataset["ANO"].apply(lambda x: 2019-x)

    dataset.ID = dataset.ID.astype("float")
    dataset.PRECO = dataset.PRECO.astype("float")
    dataset.ANO = dataset.ANO.astype("float")
    dataset.KM = dataset.KM.astype("float")
    dataset.IDADE = dataset.IDADE.astype("float")
    dataset.CIDADE = dataset.CIDADE.astype("category")
    dataset.ESTADO = dataset.ESTADO.astype("category")
    dataset.MARCA = dataset.MARCA.astype("category")
    dataset.MODELO = dataset.MODELO.astype("category")

    dataset["ESTADO"] = dataset["ESTADO"].apply(lambda x: x.upper())
    dataset["CIDADE"] = dataset["CIDADE"].apply(lambda x: x.upper())
    dataset["MARCA"] = dataset["MARCA"].apply(lambda x: x.upper())
    dataset = dataset[["ID","PRECO","ANO","IDADE","KM","CIDADE","ESTADO","MARCA","MODELO"]]
    
dataset.head()

,ID,PRECO,ANO,IDADE,KM,CIDADE,ESTADO,MARCA,MODELO
0,79293.0,18684.0,2015.0,4.0,19998.4,OWENSBORO,KY,DODGE,JourneySXT
1,190834.0,26318.0,2016.0,3.0,13563.2,CORPUS CHRISTI,TX,BMW,3
2,670383.0,25802.0,2014.0,5.0,105404.8,PORT LAVACA,TX,FORD,ExplorerLimited
3,586732.0,9899.0,2005.0,14.0,173619.2,LINDON,UT,DODGE,Dakota4WD
4,37609.0,30999.0,2016.0,3.0,13372.8,FOND DU LAC,WI,FORD,MustangFastback


## ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

**Separando a base de dados em treinamento e teste aleatoriamente**

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

In [4]:
train, test = train_test_split(dataset, test_size = 0.25, train_size = 0.75)

In [5]:
def dummify(data, column_name):
    return pd.get_dummies(data[column_name],drop_first = False, prefix = column_name)

def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    ypred = results.predict(X_cp)
    return results, ypred

In [8]:
traindummy = pd.concat([dummify(train, "ESTADO").dropna(), dummify(train, "MODELO").dropna(), dummify(train, "MARCA").dropna()], axis=1)
dt = train.drop(columns = ["ID", "ANO", "CIDADE", "ESTADO", "MARCA", "MODELO"])
final = pd.concat([dt, traindummy], axis = 1)
final.head()

,PRECO,IDADE,KM,ESTADO_ AK,ESTADO_ AL,ESTADO_ AR,ESTADO_ AZ,ESTADO_ CA,ESTADO_ CO,ESTADO_ CT,...,MARCA_SAAB,MARCA_SATURN,MARCA_SCION,MARCA_SUBARU,MARCA_SUZUKI,MARCA_TESLA,MARCA_TOYOTA,MARCA_VOLKSWAGEN,MARCA_VOLVO,MARCA_SMART
37004,7500.0,9.0,200193.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65347,33416.0,3.0,29038.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75568,8554.0,5.0,85824.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8961,18404.0,5.0,50660.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
81309,10339.0,9.0,32083.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# np.delete(final.columns.values, 0)

In [10]:
df = final

x = df[np.delete(df.columns.values, 0)]
y = df[["PRECO"]]

results = regress(x,y)

dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [30]:
results[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRECO   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     94.55
Date:                Thu, 23 May 2019   Prob (F-statistic):               0.00
Time:                        15:40:39   Log-Likelihood:            -2.4697e+05
No. Observations:               25000   AIC:                         4.972e+05
Df Residuals:                   23347   BIC:                         5.107e+05
Df Model:                        1652                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         4.797e+04    286.583    167.401      0.000    4.74e+04    4.85e+04
IDADE                        -1463.9168     19.318    -75.780      0.000   -1501.782   -1426.052
KM                              -0.0432      0.001    -51.520      0.000      -0.045      -0.042
ESTADO_ AK                     550.8854    810.505      0.680      0.497   -1037.757    2139.528
ESTADO_ AL                     840.3171    280.840      2.992      0.003     289.851    1390.783
ESTADO_ AR                    1468.2359    389.720      3.767      0.000     704.359    2232.112
ESTADO_ AZ                     566.0157    219.263      2.581      0.010     136.246     995.785
ESTADO_ CA                     674.5056    155.255      4.345      0.000     370.196     978.815
ESTADO_ CO                    1417.6979    222.515      6.371      0.000     981.554    1853.842
ESTADO_ CT                    -153.3931    311.234     -0.493      0.622    -763.433     456.646
ESTADO_ DC                    6315.0745   4838.357      1.305      0.192   -3168.422    1.58e+04
ESTADO_ DE                    1006.7097    592.264      1.700      0.089    -154.166    2167.585
ESTADO_ FL                    -338.7111    155.581     -2.177      0.029    -643.660     -33.762
ESTADO_ GA                     338.6822    186.458      1.816      0.069     -26.788     704.152
ESTADO_ HI                     925.3746    586.304      1.578      0.115    -223.820    2074.569
ESTADO_ IA                    1944.7400    389.113      4.998      0.000    1182.053    2707.427
ESTADO_ ID                     903.0077    506.702      1.782      0.075     -90.162    1896.177
ESTADO_ IL                     721.1742    188.550      3.825      0.000     351.603    1090.746
ESTADO_ IN                     293.1955    244.211      1.201      0.230    -185.475     771.866
ESTADO_ KS                     700.3476    341.511      2.051      0.040      30.964    1369.731
ESTADO_ KY                     433.8277    262.586      1.652      0.099     -80.858     948.513
ESTADO_ LA                     525.1525    325.113      1.615      0.106    -112.090    1162.395
ESTADO_ MA                     642.3687    237.054      2.710      0.007     177.728    1107.010
ESTADO_ MD                     667.9425    245.618      2.719      0.007     186.515    1149.370
ESTADO_ ME                     797.7063    762.374      1.046      0.295    -696.597    2292.009
ESTADO_ MI                     524.2200    295.453      1.774      0.076     -54.887    1103.327
ESTADO_ MN                    1249.7900    299.345      4.175      0.000     663.054    1836.526
ESTADO_ MO                     811.1615    251.275      3.228      0.001     318.646    1303.677
ESTADO_ MS                    1183.9053    381.812      3.101      0.002    

In [11]:
print(len(dataresultados), len(df))

2220 75000


In [12]:
for i in range(5):
    maiores = []
    for i in dataresultados.loc[(dataresultados["P>|t|"] > 0.10)].index.tolist():
        maiores.append(i)

    df = df.drop(columns = maiores)
    x = df[np.delete(df.columns.values, 0)]
    y = df[["PRECO"]]
    results = regress(x,y)
    dataresultados = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [13]:
print(len(dataresultados), len(df))
df.head()

1390 75000


,PRECO,IDADE,KM,ESTADO_ AK,ESTADO_ AL,ESTADO_ AR,ESTADO_ AZ,ESTADO_ CA,ESTADO_ CO,ESTADO_ DE,...,MARCA_SAAB,MARCA_SATURN,MARCA_SCION,MARCA_SUBARU,MARCA_SUZUKI,MARCA_TESLA,MARCA_TOYOTA,MARCA_VOLKSWAGEN,MARCA_VOLVO,MARCA_SMART
37004,7500.0,9.0,200193.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65347,33416.0,3.0,29038.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75568,8554.0,5.0,85824.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8961,18404.0,5.0,50660.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
81309,10339.0,9.0,32083.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
dataresultados.head()

,coef,std err,t,P>|t|,[0.025,0.975]
const,47550.0000,381.237,124.714,0.000,46800.000,48300.000
IDADE,-1429.5453,9.868,-144.869,0.000,-1448.886,-1410.204
KM,-0.0421,0.000,-92.906,0.000,-0.043,-0.041
ESTADO_ AK,1310.3842,447.035,2.931,0.003,434.197,2186.571
ESTADO_ AL,1422.6882,153.758,9.253,0.000,1121.324,1724.052


In [29]:
quanti = 0
for i in dataresultados["P>|t|"]:
    if i > 0.10:
        quanti+=1
print("Quantidade de colunas com P>|t| maiores que alpha de 10%: ", quanti)

Colunas com P>|t| maiores que alpha de 10%:  5


In [28]:
dt = test.drop(columns = ["ID", "ANO", "CIDADE", "ESTADO", "MARCA", "MODELO"])
testdummy = pd.concat([dummify(test, "ESTADO").dropna(), dummify(test, "MODELO").dropna(), dummify(test, "MARCA").dropna()], axis=1)
dfTest = pd.concat([dt, testdummy], axis = 1)

x = dfTest[np.delete(dfTest.columns.values, 0)]
y = dfTest[["PRECO"]]

results = regress(x,y)

dataresultadosTest = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]


print(len(dataresultadosTest), len(dfTest))

2221 25000


In [21]:
for i in range(5):
    maiores = []
    for i in dataresultadosTest.loc[(dataresultadosTest["P>|t|"] > 0.10)].index.tolist():
        maiores.append(i)

    dfTest = dfTest.drop(columns = maiores)
    x = dfTest[np.delete(dfTest.columns.values, 0)]
    y = dfTest[["PRECO"]]
    results = regress(x,y)
    dataresultadosTest = pd.read_html(results[0].summary().tables[1].as_html(), header=0, index_col=0)[0]

In [22]:
print(len(dataresultadosTest), len(dfTest))
dfTest.head()

1163 25000


,PRECO,IDADE,KM,ESTADO_ AR,ESTADO_ CO,ESTADO_ FL,ESTADO_ IA,ESTADO_ MN,ESTADO_ MS,ESTADO_ NC,...,MARCA_ROLLS-ROYCE,MARCA_SAAB,MARCA_SATURN,MARCA_SCION,MARCA_SUBARU,MARCA_TESLA,MARCA_TOYOTA,MARCA_VOLKSWAGEN,MARCA_VOLVO,MARCA_SMART
60127,29000.0,4.0,50643.2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7859,29977.0,5.0,68292.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57484,18500.0,5.0,64841.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38256,31500.0,2.0,37363.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28719,17973.0,6.0,112680.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
dataresultadosTest.head()

,coef,std err,t,P>|t|,[0.025,0.975]
const,45700.0000,369.970,123.520,0.000,45000.000,46400.000
IDADE,-1431.1649,17.181,-83.297,0.000,-1464.842,-1397.488
KM,-0.0436,0.001,-54.153,0.000,-0.045,-0.042
ESTADO_ AR,1040.6726,378.699,2.748,0.006,298.398,1782.947
ESTADO_ CO,909.7774,197.160,4.614,0.000,523.331,1296.224


In [24]:
quanti = 0
for i in dataresultadosTest["P>|t|"]:
    if i > 0.10:
        quanti+=1
print(quanti)

33


o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS